**Names:** Brett Nesfeder, MacAlistair Husted, Priyanka Choudhary, Shruti Kapur, Jocelyne Walker

**8:30-10:30 am section**


**Text Analytics Assignment 2: Building a crowdsourced recommendation system**


**Task A:** Extract about 5-6k reviews of 200-250 products

We've scraped Amazon.com for reviews of products returned from the keyword search "shoes."





In [ ]:
! pip install selenium 
! pip install webdriver_manager

from bs4 import BeautifulSoup
from selenium import webdriver
from pandas import DataFrame
import pandas as pd
import io
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import re
import regex 

chrome_options = Options()
chrome_options.set_headless(headless=False)
#browser = webdriver.Chrome(chrome_options=chrome_options)
browser = webdriver.Chrome(ChromeDriverManager().install())

comment_list = []
comment_title1 = []
au_list = []
comment_date = []
star_list = []
date_list = []
helpful_list = []
total_comments = 0
k = 0
i = 1
l=0
j=[]
temp_url = input('Enter Url:')
while(True):
    #temp_url = 'https://www.amazon.com/PEAK-Basketball-Streetball-Breathable-Cushioning/product-reviews/B088LXJ5Z4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    url = temp_url + "&pageNumber=%d" % i
    i = i + 1
    print(i)
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    comment_container = soup.findAll("span", {"class": "a-size-base review-text review-text-content"})
    comment_title = soup.findAll("a", {"class": "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    author = soup.findAll("span", {"class": "a-profile-name"})
    date = soup.findAll("span", {"class": "a-size-base a-color-secondary review-date"})
    star = soup.findAll("i", {"data-hook": "review-star-rating"})
    helpful = soup.findAll("div", {"data-a-expander-name": "review_comment_expander"})

    if len(comment_container) == 0:
        break

    for help in helpful:
        help_d = help.get_text()
        if "found this helpful" in help_d:
            new = help_d.split(" ")
            helpful_list.append(new[0].strip())
            # help_d[:25]
            # re.findall(r'\d+', help_d)
        else:
            helpful_list.append('0')
        #print(help_d)

    for s in star:
        star_rating = s.get_text()
        star_list.append(star_rating)


    for comment in comment_container:
        k+=1
        j.append(k)
        note_comment = comment.get_text().replace('>', '')
        #print("%d "%k, note_comment)
        comment_list.append(note_comment.strip())

    for title in comment_title:
        t = title.get_text().replace('>', '')
        #print("%d "%k, note_comment)
        comment_title1.append(t.strip())

    for c in range(2,len(author)):
        l+=1
        note_author = author[c].get_text().replace('>', '')
        #print("%d "%l, note_author)
        au_list.append(note_author)
    for a in range(2, len(date)):
        date1 = date[a].get_text().replace('on ', '')
        #print(date1)
        date_list.append(date1)

    #browser.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div/div[1]/div[2]/div[3]/div[11]/span/div/ul/li[8]/a').click()
browser.quit()

# print(df)
rating_list = []
for i in range(len(star_list)):
    rating_list.append(star_list[i][:1])

rating_list = [int(i) for i in rating_list]

temp_url = temp_url.split("//")[1]
prod_name = temp_url.split("/")[1]
prod_name = prod_name.split("-")
prod_name = ' '.join(prod_name)
product_name = []
product_name = [prod_name for product_name in range(len(comment_list))]
df = pd.DataFrame({'Number':pd.Series(j),'Date':pd.Series(date_list), 'Author':pd.Series(au_list), 'rating':pd.Series(rating_list), 'Title':pd.Series(comment_title1),'Comment':pd.Series(comment_list), 'helpful': pd.Series(helpful_list), 'product_name':pd.Series(product_name)})
df = df.dropna()
ASIN = temp_url.split("/")[3]
if ASIN[0] != 'B':
    print('Human, please check immediately!!!')
else:
    file_name = ("product_reviews".csv")
    df.to_csv(file_name)
    print('Success!')

The output file has three columns: user_rating, product_review, and product_name. It contains 8,824 reviews of 250 shoes on Amazon.com.


**Task B:** Determining customer attributes with a word frequency analysis.

In [ ]:
import pandas as pd 
import nltk
nltk.download("stopwords")
nltk.download('punkt')
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import re
from nltk.corpus import stopwords
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer 
from google.colab import files
import io
from google.colab import  drive

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['product_reviews.csv']))

df['product_review'] = df['product_review'].astype(str)
df['product_review']= df['product_review'].str.lower()

stops =set(stopwords.words("english"))
def identify_tokens(row):
    tokens = nltk.word_tokenize(row)
    # taken only words (not punctuation)
    #token_words = [w for w in tokens if w.isalpha() or w.isalnum()]
    token_words = [w for w in tokens if w.isalpha()]
    
    return token_words
df['words'] = df['product_review'].apply(identify_tokens)
def remove_stops(row):
    row
    non_stop = [w for w in row if not w in stops]
    return (non_stop)
df['words'] = df['words'].apply(remove_stops)
def rejoin_words(row):
    row
    joined_words = ( " ".join(row))
    return joined_words
df['processed'] = df['words'].apply(rejoin_words)

counter = Counter(chain.from_iterable(map(str.split, df['processed'].tolist()))) 
series = pd.Series(counter).sort_values(ascending=False)
term_freq = series.reset_index()
term_freq.columns = ['words','freq']

drive.mount('/drive')

term_freq.to_csv('/drive/My Drive/term_freq_assignment2.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Saving product_reviews.csv to product_reviews.csv
Mounted at /drive


Here are the top 30 most frequently mentioned words. 

In [ ]:
print(term_freq.iloc[:30,])

          words  freq
0         shoes  5054
1   comfortable  2880
2          shoe  2737
3         great  1983
4          size  1897
5           fit  1784
6          like  1737
7          good  1561
8          feet  1263
9          love  1248
10         wear  1246
11         pair  1152
12      running   956
13         well   908
14        would   854
15       really   849
16         look   830
17         nice   804
18      walking   790
19         feel   769
20         foot   676
21        light   674
22      support   669
23       bought   659
24          get   613
25      perfect   609
26       little   592
27        price   589
28         work   582
29          day   560


From this word frequency list, the three important attributes we have chosen are: **comfortable,** **fit,** and **support.** 

**Task C:** Perform a **similarity** analysis using cosine similarity with the three attributes specified by the customer and the reviews.

See guidelines below on how to do this step. From the output file, calculate the average similarity between each product and the preferred attributes. 
For similarity analysis, use basic cosine similarity. The script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score. While we could use word vectors, there are issues with using word vectors for recommender systems. 


In [1]:
import csv
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
import spacy
from google.colab import files
import io

nlp = spacy.load('en')

warnings.filterwarnings("ignore", category = DeprecationWarning)

uploaded = files.upload()
attributes = pd.read_csv(io.BytesIO(uploaded['attributes.txt']), header=None)

attributes = attributes.iloc[:,0].str.cat(sep=" ")
print(attributes)

Saving attributes.txt to attributes.txt
comfortable fit support


In [2]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['product_reviews.csv']))

Saving product_reviews.csv to product_reviews.csv


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import  drive
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer 


cos_similarity = df[["product_name","product_review"]].copy()
cos_similarity['similarity'] =''
#print(cos_similarity)

print("Number of rows with null values:")
print(df.isnull().sum().sum())
df = df.dropna()

for index, row in df.iterrows():
    documents = [attributes, row[1]]
    count_vectorizer = CountVectorizer(stop_words="english")
    sparse_matrix = count_vectorizer.fit_transform(documents)
    doc_term_matrix = sparse_matrix.todense()
    temp_df = pd.DataFrame(doc_term_matrix,
                           columns = count_vectorizer.get_feature_names(),
                           index = ['x','y'])
    cos_similarity.at[index,'similarity'] = cosine_similarity(temp_df,temp_df)[0,1]

#print(cos_similarity)

drive.mount('/drive')
cos_similarity.to_csv('/drive/My Drive/cos_similarity_individual_reviews.csv')

Number of rows with null values:
0
Mounted at /drive


In [5]:
cos_similarity['similarity'] = pd.to_numeric(cos_similarity['similarity'])

product_level_similarity = cos_similarity.groupby('product_name')['similarity'].mean()

merged = pd.merge(cos_similarity, product_level_similarity, on='product_name', how = 'left')

cosine_merged = pd.DataFrame(merged).iloc[:,[0,1,3]]

drive.mount('/drive')
cosine_merged.to_csv('/drive/My Drive/cos_similarity_product_avg.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [8]:
pd.set_option('display.max_columns', 3)
pd.set_option('display.width', 1000)

cosine_merged['similarity_y'] = pd.to_numeric(cosine_merged['similarity_y'])
grouped = cosine_merged.iloc[:,[0,2]].copy()

taskC = (grouped.groupby(by ="product_name")['similarity_y'].mean())
taskC = taskC.sort_values(ascending=False)
taskC = taskC.rename("cosine_similarity")
print('Top three cosine similarity products')
print(taskC[:3,])

print('\nCosine similarity output header')
print(pd.DataFrame(cos_similarity[:10]))


Top three cosine similarity products
product_name
XINBANG Breathable Sneakers Athletic Lightweight    0.263887
Dockers Mens Braxton Casual Oxford                  0.235359
New Balance Mens Tour Golf                          0.204439
Name: cosine_similarity, dtype: float64

Cosine similarity output header
                                   product_name                                     product_review  similarity
0  Orthofeet Diabetic Support Athletic Sneakers  The Whitney Lace-Up Oxford is the only shoe I ...    0.000000
1  Orthofeet Diabetic Support Athletic Sneakers  Bought these for my wife with bunions and ankl...    0.000000
2  Orthofeet Diabetic Support Athletic Sneakers  These shoes are definitely not a fashion state...    0.195180
3  Orthofeet Diabetic Support Athletic Sneakers  At 35 with horrible foot problems and wear cus...    0.084215
4  Orthofeet Diabetic Support Athletic Sneakers  I have bunions adn orthotics, so finding shoes...    0.105409
5  Orthofeet Diabetic Suppor

**Task D:** Feature level sentiment analysis 

**Strategy 1:** Parsed through reviews by pulling 3 words before and 3 words after each attribute occurrence and computing the sentiment of that phrase associated with that attribute.

In [37]:
attributes = ['comfortable','fit','support']

def sentiment1(item):
    if attributes[0] in item:
        att_index = item.index(attributes[0])
        lower_bound = max(att_index - 3, 0)
        upper_bound = min(att_index + 3, len(item))
        text = item[lower_bound:upper_bound+1]
        return vader(' '.join(text))

def sentiment2(item):
    if attributes[1] in item:
        att_index = item.index(attributes[1])
        lower_bound = max(att_index - 3, 0)
        upper_bound = min(att_index + 3, len(item))
        text = item[lower_bound:upper_bound+1]
        return vader(' '.join(text))

def sentiment3(item):
    if attributes[2] in item:
        att_index = item.index(attributes[2])
        lower_bound = max(att_index - 3, 0)
        upper_bound = min(att_index + 3, len(item))
        text = item[lower_bound:upper_bound+1]
        return vader(' '.join(text))

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def vader(item):
  return analyser.polarity_scores(item)['compound']

In [47]:
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize as tokenize
from google.colab import files
import io
nltk.download('vader_lexicon')


uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['product_reviews.csv']))

df.dropna(subset = ['product_review'], inplace=True)
df = df.reset_index(drop=True)
df['Comfortable sentiment'] = 0
df['Fit sentiment'] = 0
df['Support sentiment'] = 0
df.head()

for i in range(len(df)):
    text = df.loc[i,'product_review']

    text1 = text.split(' ')
    parsed_text = ''
    if 'comfortable' in text1:

        s_score = sentiment1(text1)
        df.loc[i,'Comfortable sentiment'] = s_score
    else:
        df.loc[i,'Comfortable sentiment'] = np.nan

    parsed_text = ''
    if 'fit' in text1:
        
        s_score = sentiment2(text1)
        df.loc[i,'Fit sentiment']+= s_score
    else:
        df.loc[i,'Fit sentiment'] = np.nan
    parsed_text = ''
    if 'support' in text1:

        s_score = sentiment3(text1)
        df.loc[i,'Support sentiment'] = s_score
    else:
        df.loc[i,'Support sentiment'] = np.nan
#df[df == 0] <- np.nan
df.head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Saving product_reviews.csv to product_reviews (19).csv


,user_rating,product_review,...,Fit sentiment,Support sentiment
0,5.0,The Whitney Lace-Up Oxford is the only shoe I ...,...,NaN,NaN
1,5.0,Bought these for my wife with bunions and ankl...,...,NaN,NaN
2,5.0,These shoes are definitely not a fashion state...,...,0.5574,NaN
3,5.0,At 35 with horrible foot problems and wear cus...,...,NaN,NaN
4,5.0,"I have bunions adn orthotics, so finding shoes...",...,NaN,NaN


In [48]:
df.groupby(by=["product_name"])[["Comfortable sentiment", "Fit sentiment", "Support sentiment"]].mean()

,Comfortable sentiment,Fit sentiment,Support sentiment
product_name,,,
361 Degrees Womens Strata Peacoat,0.538020,0.368400,0.705167
361 Degrees Womens Training Casual,NaN,NaN,NaN
361 Sensation Running White Black,NaN,0.781150,0.796400
361 Womens Strata Running Sneaker,0.599400,NaN,0.680800
AEONSHI Athletic Air Cushioning Breathable Sneakers,0.717700,0.510450,NaN
...,...,...,...
konhill Womens Mesh Walking Shoes,0.625438,0.707808,0.527367
poemlady Lightweight Sneakers Breathable Athletic,0.789867,0.799400,0.836000
poemlady Walking Lightweight Breathable Sneakers,0.632108,0.666140,-0.090450


Because this parser uses a stripped version of text to make search possible, we instead used the Strategy 2 of tokenizing sentences to create our recommendations. 

**Strategy 2:** Parsed through product reviews by tokenizing sentences 

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize as tokenize
from google.colab import files
import io

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['product_reviews.csv']))

df.dropna(subset = ['product_review'], inplace=True)
df = df.reset_index(drop=True)

df['Comfortable sentiment'] = 0
df['Fit sentiment'] = 0
df['Support sentiment'] = 0
df.head()


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Saving product_reviews.csv to product_reviews (2).csv


,user_rating,product_review,product_name,Comfortable sentiment,Fit sentiment,Support sentiment
0,5.0,The Whitney Lace-Up Oxford is the only shoe I ...,Orthofeet Diabetic Support Athletic Sneakers,0,0,0
1,5.0,Bought these for my wife with bunions and ankl...,Orthofeet Diabetic Support Athletic Sneakers,0,0,0
2,5.0,These shoes are definitely not a fashion state...,Orthofeet Diabetic Support Athletic Sneakers,0,0,0
3,5.0,At 35 with horrible foot problems and wear cus...,Orthofeet Diabetic Support Athletic Sneakers,0,0,0
4,5.0,"I have bunions adn orthotics, so finding shoes...",Orthofeet Diabetic Support Athletic Sneakers,0,0,0


In [19]:
import nltk
nltk.download('vader_lexicon')

vader = SentimentIntensityAnalyzer()
for i in range(len(df)):
    text = df.loc[i,'product_review']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [20]:
nltk.download('punkt')

vader = SentimentIntensityAnalyzer()
for i in range(len(df)):
    sentence = df.loc[i,'product_review']
    sent_token = tokenize(sentence)
    c_comfort = []
    c_fit = []
    c_support = []
    for sub_sent in sent_token:
        if 'comfortable' in sub_sent.lower():
            c_score = vader.polarity_scores(sub_sent).get('compound')
            c_comfort.append(c_score)
        if 'fit' in sub_sent.lower():
            f_score = vader.polarity_scores(sub_sent).get('compound')
            c_fit.append(f_score)
        if 'support' in sub_sent.lower():
            s_score = vader.polarity_scores(sub_sent).get('compound')
            c_support.append(s_score)
    if len(c_comfort) == 0:
        df.loc[i,'Comfortable sentiment'] = None
    else:
        df.loc[i,'Comfortable sentiment'] = np.mean(np.array(c_comfort))
    if len(c_fit) == 0:
        df.loc[i,'Fit sentiment'] = None
    else:
        df.loc[i,'Fit sentiment'] = np.mean(np.array(c_fit))
    if len(c_support) == 0:
        df.loc[i,'Support sentiment'] = None
    else:
        df.loc[i,'Support sentiment'] = np.mean(np.array(c_support))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
df.groupby(by=["product_name"])[["Comfortable sentiment", "Fit sentiment", "Support sentiment"]].mean()

,Comfortable sentiment,Fit sentiment,Support sentiment
product_name,,,
361 Degrees Womens Strata Peacoat,0.591400,0.403750,0.753725
361 Degrees Womens Training Casual,NaN,NaN,NaN
361 Sensation Running White Black,0.714533,0.797700,0.852200
361 Womens Strata Running Sneaker,0.599400,0.790100,0.680800
AEONSHI Athletic Air Cushioning Breathable Sneakers,0.788133,0.525920,NaN
...,...,...,...
konhill Womens Mesh Walking Shoes,0.670268,0.772434,0.645917
poemlady Lightweight Sneakers Breathable Athletic,0.682683,0.366400,0.836000
poemlady Walking Lightweight Breathable Sneakers,0.689489,0.514863,0.155760


In [22]:
grouped = df.groupby(by=["product_name"])[["Comfortable sentiment", "Fit sentiment", "Support sentiment"]].mean()
col = grouped.loc[: , "Comfortable sentiment":"Support sentiment"]
grouped['attribute_sentiment_mean'] = col.mean(axis=1)
grouped = grouped.sort_values(by='attribute_sentiment_mean',ascending=False)
grouped[:3]
taskD = grouped.iloc[:,3]

**Task E:** Assume an evaluation score for each shoe = average similarity score + average feature sentiment score. 

Now **recommend 3 products** to the customer.

In [23]:
pd.set_option('display.max_columns', 4)
product_ratings = df.iloc[:,[0,2]].groupby(by='product_name')['user_rating'].mean()

taskE = pd.merge(taskC,taskD, left_index = True, right_index = True)
taskE = pd.merge(taskE,product_ratings,left_index=True,right_index=True)
col = taskE.loc[: , "cosine_similarity":"attribute_sentiment_mean"]
taskE['eval_score'] = col.sum(axis=1)
taskE = taskE.sort_values(by='eval_score',ascending=False)
print('Product Recommendations')
print(taskE.iloc[:3,:])


taskE = pd.merge(taskC,taskD, left_index = True, right_index = True)
taskE_normalized =(taskE-taskE.mean())/taskE.std()
taskE_normalized = pd.merge(taskE_normalized,product_ratings,left_index=True,right_index=True)
col = taskE_normalized.loc[: , "cosine_similarity":"attribute_sentiment_mean"]
taskE_normalized['eval_score'] = col.sum(axis=1)
taskE_normalized = taskE_normalized.sort_values(by='eval_score',ascending=False)
print('\nNormalized Recommendations')
print(taskE_normalized.iloc[:3,:])

Product Recommendations
                                                    cosine_similarity  attribute_sentiment_mean  user_rating  eval_score
product_name                                                                                                            
XINBANG Breathable Sneakers Athletic Lightweight             0.263887                  0.918250     4.333333    1.182137
FUJEAK Breathable Athletic Comfortable Lightweight           0.192515                  0.775347     4.500000    0.967862
Dockers Mens Braxton Casual Oxford                           0.235359                  0.703374     4.571429    0.938733

Normalized Recommendations
                                                    cosine_similarity  attribute_sentiment_mean  user_rating  eval_score
product_name                                                                                                            
XINBANG Breathable Sneakers Athletic Lightweight             3.521868                  2.412297     4

The three products we recommend are:


*   XINBANG Breathable Sneakers Athletic Lightweight
*   FUJEAK Breathable Athletic Comfortable Lightweight
*   Dockers Mens Braxton Casual Oxford

Even when normalizing the cosine similarity and attribute sentiment means, these are the top three products with sentiment and cosine similarity forming the evaluation score.




**Task F:** How would your recommendation chance if you use word vectors (the spaCy package with pre-trained word vectors) instead of plain vanilla bag-of-words cosine similarity?

In [24]:
!python -m spacy download en_core_web_md
import csv
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
import spacy
from google.colab import files
import io
import en_core_web_md
from sklearn.feature_extraction.text import CountVectorizer

nlp = en_core_web_md.load()

warnings.filterwarnings("ignore", category = DeprecationWarning)

uploaded = files.upload()
attributes = pd.read_csv(io.BytesIO(uploaded['attributes.txt']), header=None)

attributes = list(line.strip() for line in open('attributes.txt'))
attributes = " ".join(attributes)
#attributes = attributes.iloc[:,0].str.cat(sep=" ")
print(attributes)

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=92b9daaf0f578cdb15ba9054670287f513fb9b83b5983eabe06ee9438bd0f7bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-hrtt3lr3/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


Saving attributes.txt to attributes (1).txt
comfortable fit support


In [25]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['product_reviews.csv']))

Saving product_reviews.csv to product_reviews (6).csv


In [26]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')

spacy_similarity = df[["product_name", "product_review"]].copy()
spacy_similarity['similarity'] = ''
#spacy_similarity

df = df.dropna()
print("Number of rows with null values:")
print(df.isnull().sum().sum())

df['product_review'] =df['product_review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

for index,row in df.iterrows():
   doc1 = nlp(attributes)
   doc2 = nlp(row[1])
   #print(doc1, doc1.similarity(doc2))
   spacy_similarity.at[index,'similarity'] = doc1.similarity(doc2)

print(spacy_similarity)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Number of rows with null values:
0


/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

                                           product_name                                     product_review similarity
0          Orthofeet Diabetic Support Athletic Sneakers  The Whitney Lace-Up Oxford is the only shoe I ...   0.698381
1          Orthofeet Diabetic Support Athletic Sneakers  Bought these for my wife with bunions and ankl...   0.603501
2          Orthofeet Diabetic Support Athletic Sneakers  These shoes are definitely not a fashion state...   0.708569
3          Orthofeet Diabetic Support Athletic Sneakers  At 35 with horrible foot problems and wear cus...   0.672049
4          Orthofeet Diabetic Support Athletic Sneakers  I have bunions adn orthotics, so finding shoes...   0.694301
...                                                 ...                                                ...        ...
8823  GGchenhao Breathable Lightweight Comfortable S...  They are very comfortable and the color is great.   0.758241
8824  GGchenhao Breathable Lightweight Comfortable S... 

In [ ]:
spacy_similarity['similarity'] = pd.to_numeric(spacy_similarity['similarity'])

product_level_similarity = spacy_similarity.groupby('product_name')['similarity'].mean()

merged = pd.merge(cos_similarity, product_level_similarity, on='product_name', how = 'left')

spacy_merged = pd.DataFrame(merged).iloc[:,[0,1,3]]

drive.mount('/drive')
spacy_merged.to_csv('/drive/My Drive/spacy_similarity_product_avg.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
merged = pd.merge(cos_similarity, spacy_similarity, on = ('product_name','product_review'), suffixes = ("_cosine","_spacy"))
merged = pd.merge(merged, cosine_merged, on = ('product_name','product_review'))
merged = pd.merge(merged, spacy_merged, on = ('product_name','product_review'), suffixes = ("_cosine_avg","_spacy_avg"))

print(merged.columns)

drive.mount('/drive')
merged.to_csv('/drive/My Drive/similarity_comparison.csv')

Index(['product_name', 'product_review', 'similarity_cosine', 'similarity_spacy', 'similarity_y_cosine_avg', 'similarity_y_spacy_avg'], dtype='object')
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
spacy_merged['similarity_y'] = pd.to_numeric(spacy_merged['similarity_y'])
grouped = spacy_merged.iloc[:,[0,2]].copy()

taskF = (grouped.groupby(by ="product_name")['similarity_y'].mean())
taskF = taskF.sort_values(ascending=False)
taskF = taskF.rename("spacy_similarity")
print('Three products with highest spacy similarity')
print(taskF[:3,])

Three products with highest spacy similarity
product_name
361 Womens Strata Running Sneaker        0.724505
Mizuno Womens WAVEKNIT Running Arctic    0.720614
Puno King Leather Fashion Sneaker        0.714885
Name: spacy_similarity, dtype: float64


In [ ]:
pd.set_option('display.max_columns', None)
product_ratings = df.iloc[:,[0,2]].groupby(by='product_name')['user_rating'].mean()

taskE = pd.merge(taskF,taskD, left_index = True, right_index = True)
taskE = pd.merge(taskE,product_ratings,left_index=True,right_index=True)
col = taskE.loc[: , "spacy_similarity":"attribute_sentiment_mean"]
taskE['eval_score'] = col.sum(axis=1)
taskE = taskE.sort_values(by='eval_score',ascending=False)
print('Product Recommendations Using SpaCy')
print(taskE.iloc[:3,:])

taskE = pd.merge(taskF,taskD, left_index = True, right_index = True)
taskE_normalized =(taskE-taskE.mean())/taskE.std()
taskE_normalized = pd.merge(taskE_normalized,product_ratings,left_index=True,right_index=True)
col = taskE_normalized.loc[: , "spacy_similarity":"attribute_sentiment_mean"]
taskE_normalized['eval_score'] = col.sum(axis=1)
taskE_normalized = taskE_normalized.sort_values(by='eval_score',ascending=False)
print('\nNormalized Recommendations Using SpaCy')
print(taskE_normalized.iloc[:3,:])

Product Recommendations Using SpaCy
                                                    spacy_similarity  attribute_sentiment_mean  user_rating  eval_score
product_name                                                                                                           
XINBANG Breathable Sneakers Athletic Lightweight            0.626213                  0.918250     4.333333    1.544463
FUJEAK Breathable Athletic Comfortable Lightweight          0.708755                  0.775347     4.500000    1.484102
SKDOIUL Running Breathable Athletic Sneakers                0.661973                  0.815525     5.000000    1.477498

Normalized Recommendations Using SpaCy
                                                    spacy_similarity  attribute_sentiment_mean  user_rating  eval_score
product_name                                                                                                           
FUJEAK Breathable Athletic Comfortable Lightweight          1.190661                

The three products recommended by the spacy recommendations are:



*   XINBANG Breathable Sneakers Athletic Lightweight
*   FUJEAK Breathable Athletic Comfortable Lightweight
*   SKDOIUL Running Breathable Athletic Sneakers

It is interesting to note that once the values were normalized, the third recommended product changed to Soulsfeng Breathable Athletic Training Sneakers.


The top 2 products, XINBANG and FUJEAK sneakers, were also the top 2 recommended products by cosine similarity.



In [ ]:
import pandas as pd
product_reviews = pd.read_csv('product_reviews.csv')
def findcomfortable(review):
    review = str(review)
    review_list = review.split(' ')
    word = 'comfortable'
    if word in review_list:
        return 1 
    else: 
        return 0
def findfit(review):
    review = str(review)
    review_list = review.split(' ')
    word = 'fit'
    if word in review_list:
        return 1 
    else: 
        return 0
def findsupport(review):
    review = str(review)
    review_list = review.split(' ')
    word = 'support'
    if word in review_list:
        return 1 
    else: 
        return 0

product_review_count = product_reviews.groupby(by = 'product_name')['product_review'].count()    
product_reviews['comfortable']= product_reviews['product_review'].apply(findcomfortable)
product_reviews['fit']= product_reviews['product_review'].apply(findfit)
product_reviews['support']= product_reviews['product_review'].apply(findsupport)

product_support_c = product_reviews.groupby(by = 'product_name')['comfortable'].sum()
product_support_f = product_reviews.groupby(by = 'product_name')['fit'].sum()
product_support_s = product_reviews.groupby(by = 'product_name')['support'].sum()

product_support_c= product_support_c.reset_index()
product_support_f= product_support_f.reset_index()
product_support_s= product_support_s.reset_index()

product_review_count = product_review_count.reset_index()

product_review_count['comfortable'] = product_support_c['comfortable']
product_review_count['fit'] = product_support_f['fit']
product_review_count['support'] = product_support_s['support']

product_review_count['comfort_percentage'] = product_support_c['comfortable']/product_review_count['product_review']
product_review_count['fit_percentage'] = product_support_f['fit']/product_review_count['product_review']
product_review_count['support_percentage'] = product_support_s['support']/product_review_count['product_review']

def percentify(decimal):
    percentage = "{:.0%}".format(decimal)
    return percentage
product_review_count['comfort_percentage']= product_review_count['comfort_percentage'].apply(percentify)
product_review_count['fit_percentage']= product_review_count['fit_percentage'].apply(percentify)
product_review_count['support_percentage']= product_review_count['support_percentage'].apply(percentify)

prod_1 = product_review_count.loc[product_review_count['product_name'] == 'XINBANG Breathable Sneakers Athletic Lightweight']
prod_2 = product_review_count.loc[product_review_count['product_name'] == 'FUJEAK Breathable Athletic Comfortable Lightweight']
prod_3 = product_review_count.loc[product_review_count['product_name'] == 'SKDOIUL Running Breathable Athletic Sneakers']
prod_4 = product_review_count.loc[product_review_count['product_name'] == 'Dockers Mens Braxton Casual Oxford']


print(prod_1)
print('\n',prod_2)
print('\n',prod_3)
print('\n',prod_4)

                                         product_name  product_review  comfortable  fit  support comfort_percentage fit_percentage support_percentage
222  XINBANG Breathable Sneakers Athletic Lightweight               3            2    2        0                67%            67%                 0%

                                          product_name  product_review  comfortable  fit  support comfort_percentage fit_percentage support_percentage
55  FUJEAK Breathable Athletic Comfortable Lightwe...              20           14    2        0                70%            10%                 0%

                                      product_name  product_review  comfortable  fit  support comfort_percentage fit_percentage support_percentage
176  SKDOIUL Running Breathable Athletic Sneakers               2            2    0        0               100%             0%                 0%

                           product_name  product_review  comfortable  fit  support comfort_percentage f

As the top two products were the same in cosine and spacy similarity, we did not see a stark difference in the difference in recommended products. We do see that the top product, XINBANG, only has three product reviews. Two thirds of these mentioned comfort and fit, but support was not mentioned in any of them.

Of all the products recommended by Cosine and Spacy similarity plus the Attribute Sentiment (together forming the Evaluation score), support was not directly mentioned in any of them. This might mean that the spaCy similarity used other words similar to support to measure similarity. However, the SKODOIL, the one additional product recommended by spaCy similarity, did have both of its reviews mention "comfort" directly. We did see a dip in the direct mention of fit percentage, as spacy uses the word vector approach instead.

All the products recommended by Cosine similarity, XINBANG, FUJEAK, and Dockers, at least had some mention of comfort and fit directly in their reviews. 


**Task G:** How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings (e.g., from the Amazon website) to answer this question.

In [ ]:
# three highest rated products from the dataset

# first, based on cosine similarity 
taskE = pd.merge(taskC,taskD, left_index = True, right_index = True)
taskE = pd.merge(taskE,product_ratings,left_index=True,right_index=True)
col = taskE.loc[: , "cosine_similarity":"attribute_sentiment_mean"]
taskE['eval_score'] = col.sum(axis=1)
print(taskE.sort_values(by=['user_rating','eval_score'],ascending=False).iloc[:10,:])

#filter out where eval_score = 0

prod_1 = product_review_count.loc[product_review_count['product_name'] == '361 Womens Strata Running Sneaker']
prod_2 = product_review_count.loc[product_review_count['product_name'] == 'FitVille Running Sneaker Breathable Climbing']
prod_3 = product_review_count.loc[product_review_count['product_name'] == 'SKDOIUL Running Breathable Athletic Sneakers']
prod_4 = product_review_count.loc[product_review_count['product_name'] == 'APRILSPRING Wear Resistant Anti Skid Absorption Sneakers']

print('\n',prod_3)
print('\n',prod_1)
print('\n',prod_2)
print('\n',prod_4)

                                                    cosine_similarity  attribute_sentiment_mean  user_rating  eval_score
product_name                                                                                                            
SKDOIUL Running Breathable Athletic Sneakers                 0.109219                  0.815525     5.000000    0.924744
361 Womens Strata Running Sneaker                            0.142109                  0.690100     5.000000    0.832209
FitVille Running Sneaker Breathable Climbing                 0.090607                  0.656842     5.000000    0.747449
APRILSPRING Wear Resistant Anti Skid Absorption...           0.093659                  0.556300     5.000000    0.649959
NIKE Mens 316408 161 Size 11 5                               0.000000                       NaN     5.000000    0.000000
Goodyear Mens Clutch Racer Sneaker                           0.000000                  0.000000     5.000000    0.000000
adidas Ultraboost Sneaker Black 

It's interesting to note that the SKDOIL Running Breathable Sneakers were one of the products with a perfect user rating (5.00) that were also mentioned by our spaCy similarity approach. 

The other products that might have been recommended just on perfect 5 star ratings are:

*   361 Womens Strata Running Sneaker                            
*   FitVille Running Sneaker Breathable Climbing                 
*   APRILSPRING Wear Resistant Anti Skid Absorption

These products are less likely to meet the requirements of the user looking for recommendations. Their cosine similarity is much less for the attribute values, and their attribute sentiment mean is significantly less than the sentiment means for the other products. This may mean that the reviewers for these products, even though they gave 5 stars, were less enthusiastic in their sentiment for the attributes. We also see that fit and support are much less likely to be mentioned in these reviews compared to our previously recommended products. 

